In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [2]:
root = "/Users/kshitij1/Documents/GitHub/Using-Twitter-for-Elections-Analysis/Datasets/Media"

In [3]:
files = os.listdir(root)

In [4]:
df = pd.read_csv(root+"/"+files[0])

In [5]:
tweets = df['tweet']

In [6]:
aap_list = ["aap","#AAP","@AamAadmiParty","kejriwal","#ArvindKejriwal","@AAPPunjab","#KejriwalVsAll","@ArvindKejriwal","#AAPdePaap","@msisodia","@BhagwantMann"]
inc_list = ["congress","INC","@INCIndia","@RahulGandhi","Rahul Gandhi","@INCPunjab","@priyankagandhi","@INCUttarPradesh"]
bsp_list = ["@Mayawati","bsp","@BSPIndia","bahujan samaj","mayawati","@AnandAkash_BSP","@satishmisrabsp"]
samajwadi_list = ["@yadavakhilesh","samajwadi","@samajwadiparty","shivpal","#MulayamSinghYadav"]
bjp_list = ["bjp","@BJP4India","@narendramodi","#BJPwinningUP","@AmitShah","@myogiadityanath","yogi","@BJP4TamilNadu","@BJP4UP","@JPNadda","@CHARANJITCHANNI","channi"]

In [7]:
# lower case all text in list
aap_list = [x.lower() for x in aap_list]
# print(aap_list)
inc_list = [x.lower() for x in inc_list]
bsp_list = [x.lower() for x in bsp_list]
samajwadi_list = [x.lower() for x in samajwadi_list]
bjp_list = [x.lower() for x in bjp_list]

In [8]:
for i,row in tqdm(df.iterrows(),total = len(df.index)):
    tweet = row['tweet'].lower()
    words = tweet.split()
    # words = tweets[i].split()
    # print(words)
    for word in words:
        if word in aap_list:
            df.loc[i,'party'] = 'aap'
            break
        elif word in inc_list:
            df.loc[i,'party'] = 'inc'
            break
        elif word in bsp_list:
            df.loc[i,'party'] = 'bsp'
            break
        elif word in samajwadi_list:
            df.loc[i,'party'] = 'samajwadi'
            break
        elif word in bjp_list:
            df.loc[i,'party'] = 'bjp'
            break
        else:
            df.loc[i,'party'] = 'other'
            break

100%|██████████| 20950/20950 [00:08<00:00, 2513.91it/s]


In [9]:
import pandas as pd
from sentiment import load_models,predict, predict_vader
from preprocess import preprocessList
import os
import numpy as np
from textblob import TextBlob
from nltk.util import ngrams
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from tqdm import tqdm

In [10]:
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity
def getAnalysis(score):
    if score < 0:
        return 'negative'
    elif score==0:
        return 'neutral'
    else:
        return 'positive'

In [11]:
df = df[df['party']!='other']

In [14]:
sentiment_dir = "/Users/kshitij1/Documents/GitHub/Using-Twitter-for-Elections-Analysis/Media Bias/Sentiments/"

In [16]:
parties = ["aap","bjp","inc","samajwadi","bsp"]

In [18]:
for file in tqdm(files):
    df = pd.read_csv(root+"/"+file)
    tweets = df['tweet']
    for i,row in df.iterrows():
        tweet = row['tweet'].lower()
        words = tweet.split()
        # words = tweets[i].split()
        # print(words)
        for word in words:
            if word in aap_list:
                df.loc[i,'party'] = 'aap'
                break
            elif word in inc_list:
                df.loc[i,'party'] = 'inc'
                break
            elif word in bsp_list:
                df.loc[i,'party'] = 'bsp'
                break
            elif word in samajwadi_list:
                df.loc[i,'party'] = 'samajwadi'
                break
            elif word in bjp_list:
                df.loc[i,'party'] = 'bjp'
                break
            else:
                df.loc[i,'party'] = 'other'
                break

    for party in parties:
        tempdf = df[df['party']==party]
        tempdf.to_csv(party+".csv",index=False)

 40%|████      | 2/5 [00:15<00:22,  7.34s/it]/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
100%|██████████| 5/5 [00:55<00:00, 11.12s/it]


In [17]:
for file in tqdm(files):
    df = pd.read_csv(root+"/"+file)
    tweets = df['tweet']
    for i,row in df.iterrows():
        tweet = row['tweet'].lower()
        words = tweet.split()
        # words = tweets[i].split()
        # print(words)
        for word in words:
            if word in aap_list:
                df.loc[i,'party'] = 'aap'
                break
            elif word in inc_list:
                df.loc[i,'party'] = 'inc'
                break
            elif word in bsp_list:
                df.loc[i,'party'] = 'bsp'
                break
            elif word in samajwadi_list:
                df.loc[i,'party'] = 'samajwadi'
                break
            elif word in bjp_list:
                df.loc[i,'party'] = 'bjp'
                break
            else:
                df.loc[i,'party'] = 'other'
                break
    
    for party in parties:
        tempdf = df[df['party']==party]
        tweets = tempdf['tweet']
        vectoriser, BNBmodel = load_models()
        sentiments = predict(vectoriser,BNBmodel,preprocessList(tweets))['sentiment']
        fname = file.split('.')[0] + f"_{party}"
        np.save(sentiment_dir+fname+"_BNB.npy",sentiments)
        subjectivity_list = [getSubjectivity(tweet) for tweet in tweets]
        polarity_list = [getPolarity(tweet) for tweet in tweets]
        np.save(sentiment_dir+fname+"_subjectivity.npy",subjectivity_list)
        np.save(sentiment_dir+fname+"_polarity.npy",polarity_list)
    

 40%|████      | 2/5 [00:35<00:52, 17.58s/it]/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
100%|██████████| 5/5 [01:50<00:00, 22.17s/it]
